In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.font_manager as font_manager

# funciones desarrolladas
from functions.cargar_data import *
from functions.guardar_data import *
from functions.graficas import *

Matplotlib is building the font cache; this may take a moment.


In [2]:
variables = ['DPTO', 'LOC','PERMI07', 'PERMI07_1', 'PERMI07_2', 'PERMI07_3', 'PERMI07_4']

In [118]:
censo = cargar_censo_vars(variables)

In [119]:
censo.head()

,DPTO,LOC,PERMI07,PERMI07_1,PERMI07_2,PERMI07_3,PERMI07_4
0,1,20,0,NaN,NaN,NaN,0
1,1,20,0,NaN,NaN,NaN,0
2,1,20,0,NaN,NaN,NaN,0
3,1,20,0,NaN,NaN,NaN,0
4,1,20,0,NaN,NaN,NaN,0


In [120]:
def conteo_migrantes(df):
    "Conteo de migrantes interno por departamento"
    lista_vars = ['total', 'cat_a', 'cat_b', 'cat_c']
    df[lista_vars[0]] = 1
    [df.insert(len(df.columns), lista_vars[i], 0) for i in range(1,4)]
    # destino Mvdeo.
    df.loc[(df.PERMI07 == 3) & (df.DPTO==1), lista_vars[1]] = 1
    # destino interior
    df.loc[(df.PERMI07 == 3) & (df.PERMI07_2== 1), lista_vars[2]] = 1
    # entre interior
    df.loc[(df.PERMI07 == 3) & (df.PERMI07_2!=1) & (df.DPTO!=1) & (df.PERMI07_2.isnull()==False), lista_vars[3]] = 1

    # agrupa
    df_group = df.groupby('DPTO').agg(
                {   'total': sum,
                     lista_vars[1]: sum,
                     lista_vars[2]: sum,
                     lista_vars[3]: sum
                })

    df_group['total_mig'] = df_group[lista_vars[1]] + df_group[lista_vars[2]] + df_group[lista_vars[3]]
    
    return df_group

In [121]:
migrantes = conteo_migrantes(censo)
migrantes

,total,cat_a,cat_b,cat_c,total_mig
DPTO,,,,,
1,1318755,42444,0,0,42444
2,73377,0,914,896,1810
3,520173,0,33127,8286,41413
4,84698,0,1387,1614,3001
5,123203,0,2100,3362,5462
6,57084,0,982,1589,2571
7,25050,0,378,933,1311
8,67047,0,1026,2257,3283
9,58815,0,825,1760,2585


In [122]:
# Exporta CSV
# migrantes.to_csv('tablas/migrantes_por_depto.csv')

In [123]:
censo.groupby(['PERMI07_2', 'PERMI07_3']).DPTO.count().reset_index()

,PERMI07_2,PERMI07_3,DPTO
0,1.0,20.0,57982
1,1.0,621.0,125
2,1.0,622.0,133
3,1.0,721.0,173
4,1.0,900.0,237
...,...,...,...
890,19.0,949.0,2
891,19.0,951.0,5
892,19.0,953.0,1
893,19.0,960.0,4


In [124]:
def conteo_migrantes_por_loc(df):
    "Conteo por localidad de migrantes internos con origen en Mvdeo."
    df_filtered = df.loc[(df.PERMI07 == 3) & (df.PERMI07_2== 1)].copy()
    df_filtered['CODLOC'] = df_filtered.DPTO.astype(str) + pad(df_filtered, 'LOC', 3)
    df_grouped = df_filtered.groupby(['CODLOC']).DPTO.count().reset_index()
    df_grouped.rename({'CODLOC':'loc_destino', 'DPTO':'personas'}, inplace=True, axis=1)
    
    return df_grouped.sort_values('personas', ascending=False)

In [125]:
mig_loc = conteo_migrantes_por_loc(censo)

In [126]:
# exporta
# mig_loc.to_csv('tablas/migrantes_no_mvo_por_loc.csv', index=False)
# campos = 'Integer','Integer'
# export_csvt(campos, 'tablas/migrantes_no_mvo_por_loc.csvt')

In [127]:
merge_loc = loc_decode(mig_loc)
merge_loc.head()

,loc_destino,personas,departamento,localidad
0,3828,2699,CANELONES,EL PINAR
1,3953,2660,CANELONES,LOMAS DE SOLYMAR
2,3732,2600,CANELONES,SOLYMAR
3,3221,2110,CANELONES,LAS PIEDRAS
4,16521,2014,SAN JOSE,DELTA DEL TIGRE Y VILLAS


In [128]:
# porcentaje de personas migrantes internas corresppondientes a las 7
# localidades que más reciben migrantes
print(round(merge_loc.head(7).personas.sum() / merge_loc.personas.sum() * 100, 2 ))

27.15


In [129]:
# identifica casos de AM
am = pd.read_csv('tablas/localidades_am_mvo.csv', usecols=['codloc'], dtype= {'codloc': str,})
migrantes_mvo = merge_loc.merge(am, how='left', left_on='loc_destino', right_on='codloc')
migrantes_mvo.loc[~migrantes_mvo.codloc.isnull(), 'categoria'] = 'AREA METRO'
migrantes_mvo.loc[migrantes_mvo.codloc.isnull(), 'categoria'] = migrantes_mvo.departamento

# calcula n y porcentajes
n_mig = migrantes_mvo.groupby(['categoria']).personas.sum()
percent_mig = (migrantes_mvo.groupby(['categoria']).personas.sum()/migrantes_mvo.personas.sum()*100).round(2).rename('porcentaje', axis=1)
migrantes_mvo_am= pd.concat([n_mig, percent_mig], axis=1).reset_index()

# agrega total
total = pd.DataFrame([['Total', sum(migrantes_mvo_am.personas), sum(migrantes_mvo_am.porcentaje)]], columns=['categoria', 'personas', 'porcentaje'])
migrantes_mvo_am = migrantes_mvo_am.append(total)

In [130]:
migrantes_mvo_am.loc[:, 'categoria'] = format_depto(migrantes_mvo_am, 'categoria')

migrantes_mvo_am.rename({'categoria':'Entidad geográfica'}, inplace=True, axis=1)

In [131]:
print(migrantes_mvo_am.to_markdown(index=False, tablefmt='pretty'))

Entidad geográfica      personas    porcentaje
--------------------  ----------  ------------
Área metropolitana         32750         58.45
Artigas                      894          1.6
Canelones                   1989          3.55
Cerro Largo                 1355          2.42
Colonia                     2015          3.6
Durazno                      930          1.66
Flores                       339          0.61
Florida                      886          1.58
Lavalleja                    746          1.33
Maldonado                   3830          6.84
Paysandú                    1059          1.89
Río Negro                    821          1.47
Rivera                      1626          2.9
Rocha                       1211          2.16
Salto                       1480          2.64
San José                     921          1.64
Soriano                     1117          1.99
Tacuarembó                  1329          2.37
Treinta y Tres               730          1.3
Total            